In [1]:
import geopandas as gpd
import pandas as pd
from siuba import *

from segment_speed_utils import helpers, gtfs_schedule_wrangling
from shared_utils import rt_dates, gtfs_utils_v2
import folium

In [2]:
from update_vars import analysis_date, AM_PEAK, PM_PEAK, EXPORT_PATH, GCS_FILE_PATH, PROJECT_CRS

In [3]:
import sjoin_stops_to_segments

In [4]:
analysis_date

'2024-09-18'

In [5]:
# (1) Aggregate stop times - by stop_id, find max trips in AM/PM peak
# takes 1 min
max_arrivals_by_stop = helpers.import_scheduled_stop_times(
    analysis_date,
    get_pandas = True,
).pipe(sjoin_stops_to_segments.prep_stop_times).pipe(sjoin_stops_to_segments.stop_times_aggregation_max_by_stop, analysis_date)

In [6]:
# (1) Aggregate stop times - by stop_id, find max trips in AM/PM peak
# takes 1 min
max_arrivals_by_stop_single = helpers.import_scheduled_stop_times(
    analysis_date,
    get_pandas = True,
).pipe(sjoin_stops_to_segments.prep_stop_times).pipe(
    sjoin_stops_to_segments.stop_times_aggregation_max_by_stop, analysis_date, single_route_dir=True)

In [7]:
new_hq_multi = max_arrivals_by_stop >> filter(_.am_max_trips_hr > 4, _.pm_max_trips_hr > 4) #  new HQ corridor (still multi-route)
new_ms_multi = max_arrivals_by_stop >> filter(_.am_max_trips_hr > 3, _.pm_max_trips_hr > 3) #  new major stop precursor (still multi-route)
new_hq_single = max_arrivals_by_stop_single >> filter(_.am_max_trips_hr > 4, _.pm_max_trips_hr > 4) #  new HQ corridor (single-route)
new_ms_single = max_arrivals_by_stop_single >> filter(_.am_max_trips_hr > 3, _.pm_max_trips_hr > 3) #  new major stop precursor (single-route)

## preliminary mapping

* add bus/rail/ferry MTS in orange to help decide

In [8]:
OLD_EXPORT_PATH = f"{GCS_FILE_PATH}export/{rt_dates.DATES['aug2024']}/"

In [9]:
old_stops = gpd.read_parquet(f"{OLD_EXPORT_PATH}ca_hq_transit_stops.parquet")

In [10]:
non_bus = old_stops >> filter(-_.hqta_type.isin(['hq_corridor_bus', 'major_stop_bus']))

In [11]:
feeds = (gtfs_utils_v2.schedule_daily_feed_to_gtfs_dataset_name(selected_date=analysis_date)
         >> select(_.feed_key, _.schedule_gtfs_dataset_key == _.gtfs_dataset_key)
         >> filter(_.schedule_gtfs_dataset_key.isin(max_arrivals_by_stop.schedule_gtfs_dataset_key))
        )

In [12]:
stops = gtfs_utils_v2.get_stops(selected_date=analysis_date, operator_feeds=feeds.feed_key,
                       stop_cols=['feed_key', 'stop_id'])

/opt/conda/lib/python3.9/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'GEOGRAPHY' of column 'pt_geom'
  sqlalchemy.util.warn(


In [13]:
stops = stops >> inner_join(_, feeds, on='feed_key')

In [14]:
def add_to_map(test_df, m=None, **kwargs):
    gdf = stops >> inner_join(_, test_df, on=['schedule_gtfs_dataset_key', 'stop_id'])
    m = gdf.explore(m = m, **kwargs)
    return m

In [15]:
m1 = old_stops.explore(color='blue')
m1 = non_bus.explore(m = m1, color='orange')

In [16]:
m1 = add_to_map(new_hq_multi, m=m1, color='green')

In [17]:
m1 = add_to_map(new_hq_single, m=m1, color='red')

In [18]:
folium.LayerControl().add_to(m1);

In [19]:
# m1

## major stops

In [20]:
m = old_stops.explore(color='blue')
m = non_bus.explore(m = m, color='orange')

In [21]:
m = add_to_map(new_ms_multi, m=m, color='green')

In [22]:
m = add_to_map(new_ms_single, m=m, color='red')

In [23]:
folium.LayerControl().add_to(m);

In [64]:
# m

# full pipeline check

In [25]:
new_stops = gpd.read_parquet(f"{EXPORT_PATH}ca_hq_transit_stops.parquet")

In [26]:
new_areas = gpd.read_parquet(f"{EXPORT_PATH}ca_hq_transit_areas.parquet")

In [27]:
f"{EXPORT_PATH}ca_hq_transit_areas.parquet"

'gs://calitp-analytics-data/data-analyses/high_quality_transit_areas/export/2024-09-18/ca_hq_transit_areas.parquet'

In [28]:
old_areas = gpd.read_parquet(f"{OLD_EXPORT_PATH}ca_hq_transit_areas.parquet")

In [29]:
new_stops.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 114837 entries, 0 to 114836
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype   
---  ------                --------------   -----   
 0   agency_primary        114837 non-null  object  
 1   hqta_type             114837 non-null  object  
 2   stop_id               114837 non-null  object  
 3   route_id              114837 non-null  object  
 4   hqta_details          114837 non-null  object  
 5   agency_secondary      44019 non-null   object  
 6   base64_url_primary    114837 non-null  object  
 7   base64_url_secondary  44019 non-null   object  
 8   org_id_primary        114837 non-null  object  
 9   org_id_secondary      44019 non-null   object  
 10  peak_trips            114837 non-null  int64   
 11  geometry              114837 non-null  geometry
dtypes: geometry(1), int64(1), object(10)
memory usage: 10.5+ MB


In [30]:
new_stops >> count(_.hqta_type)

,hqta_type,n
0,hq_corridor_bus,68167
1,major_stop_brt,438
2,major_stop_bus,44019
3,major_stop_ferry,50
4,major_stop_rail,2163


In [31]:
old_stops >> count(_.hqta_type)

,hqta_type,n
0,hq_corridor_bus,43110
1,major_stop_brt,264
2,major_stop_bus,16498
3,major_stop_ferry,25
4,major_stop_rail,865


In [32]:
old_stops.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 60762 entries, 0 to 60761
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   agency_primary        60762 non-null  object  
 1   hqta_type             60762 non-null  object  
 2   stop_id               60762 non-null  object  
 3   route_id              60762 non-null  object  
 4   hqta_details          60762 non-null  object  
 5   agency_secondary      16498 non-null  object  
 6   base64_url_primary    60762 non-null  object  
 7   base64_url_secondary  16498 non-null  object  
 8   org_id_primary        60762 non-null  object  
 9   org_id_secondary      16498 non-null  object  
 10  peak_trips            60762 non-null  int64   
 11  geometry              60762 non-null  geometry
dtypes: geometry(1), int64(1), object(10)
memory usage: 5.6+ MB


In [34]:
m2 = (old_stops >> filter(_.hqta_type == 'major_stop_bus')).explore(color='blue')
m2 = (new_stops >> filter(_.hqta_type == 'major_stop_bus')).explore(m = m2, color='orange')
folium.LayerControl().add_to(m2);

In [35]:
# m2

In [36]:
from calitp_data_analysis.geography_utils import CA_NAD83Albers

In [37]:
new_dissolved = new_areas.to_crs(CA_NAD83Albers).dissolve(by='hqta_type').reset_index()
new_dissolved['area'] = new_dissolved.geometry.apply(lambda x: x.area)
new_dissolved[['hqta_type', 'area']]

,hqta_type,area
0,hq_corridor_bus,5.909532e+09
1,major_stop_brt,1.280199e+08
2,major_stop_bus,4.277683e+09
3,major_stop_ferry,3.810941e+07
4,major_stop_rail,7.895734e+08


In [38]:
old_dissolved = old_areas.to_crs(CA_NAD83Albers).dissolve(by='hqta_type').reset_index()
old_dissolved['area'] = old_dissolved.geometry.apply(lambda x: x.area)
old_dissolved[['hqta_type', 'area']]

,hqta_type,area
0,hq_corridor_bus,8.536262e+09
1,major_stop_brt,1.066221e+08
2,major_stop_bus,3.639118e+09
3,major_stop_ferry,2.814088e+07
4,major_stop_rail,6.248111e+08


In [39]:
old_hq = old_dissolved >> filter(_.hqta_type == 'major_stop_bus')

In [40]:
new_hq = new_dissolved >> filter(_.hqta_type == "major_stop_bus")

In [41]:
only_old = old_hq.overlay(new_hq, how='difference')

In [42]:
only_new = new_hq.overlay(old_hq, how='difference')

In [43]:
# only_new.explore()

In [44]:
# only_old.explore()

In [45]:
import intake

In [46]:
catalog = intake.open_catalog("*.yml")

In [47]:
hqta_points = catalog.hqta_points.read().to_crs(PROJECT_CRS)

In [48]:
hqta_points >> filter(_.stop_id=='62965')

,agency_primary,hqta_type,stop_id,route_id,hqta_details,agency_secondary,base64_url_primary,base64_url_secondary,org_id_primary,org_id_secondary,peak_trips,geometry
106072,Santa Clara Valley Transportation Authority,hq_corridor_bus,62965,64B,corridor_other_stop,None,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,None,recC5CT95EufmQCXr,None,1,POINT (-169331.147 -80088.000)
108176,Santa Clara Valley Transportation Authority,major_stop_bus,62965,64B,intersection_2_bus_routes_same_operator,Santa Clara Valley Transportation Authority,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,recC5CT95EufmQCXr,recC5CT95EufmQCXr,1,POINT (-169331.147 -80088.000)


## Counting rail/ferry stops

* Prior refactors have us keeping a row per stop_id x route_id, will keep
* did scan and remove a few ferry stops without bus/rail (Havasu, other Angel Island operator)

In [49]:
# (new_stops >> filter(_.hqta_type == 'major_stop_ferry')).explore()

In [50]:
# (old_stops >> filter(_.hqta_type == 'major_stop_ferry')).explore()

In [51]:
(old_stops >> filter(_.hqta_type == 'major_stop_ferry')) >> count(_.agency_primary)

,agency_primary,n
0,Chemehuevi Indian Tribe,2
1,"Golden Gate Bridge, Highway and Transportation...",5
2,San Diego Metropolitan Transit System,3
3,San Francisco Bay Area Water Emergency Transit...,13
4,Treasure Island Community Development,2


In [52]:
(new_stops >> filter(_.hqta_type == 'major_stop_ferry')) >> count(_.agency_primary)

,agency_primary,n
0,Angel Island-Tiburon Ferry Company,2
1,Chemehuevi Indian Tribe,2
2,Flagship Cruises and Events Inc.,3
3,"Golden Gate Bridge, Highway and Transportation...",9
4,San Diego International Airport,3
5,San Diego Metropolitan Transit System,3
6,San Francisco Bay Area Water Emergency Transit...,26
7,Treasure Island Community Development,2


In [53]:
old_stops >> filter(_.hqta_type == 'major_stop_ferry', _.agency_primary.str.contains('Golden'))

,agency_primary,hqta_type,stop_id,route_id,hqta_details,agency_secondary,base64_url_primary,base64_url_secondary,org_id_primary,org_id_secondary,peak_trips,geometry
26181,"Golden Gate Bridge, Highway and Transportation...",major_stop_ferry,43000,LSSF,major_stop_ferry_single_operator,None,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,None,recoX7qMhlPrgfuz3,None,3,POINT (-122.39243 37.79592)
26182,"Golden Gate Bridge, Highway and Transportation...",major_stop_ferry,43003,SSSF,major_stop_ferry_single_operator,None,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,None,recoX7qMhlPrgfuz3,None,2,POINT (-122.47755 37.85617)
26183,"Golden Gate Bridge, Highway and Transportation...",major_stop_ferry,43004,LSSF,major_stop_ferry_single_operator,None,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,None,recoX7qMhlPrgfuz3,None,3,POINT (-122.50835 37.94471)
26184,"Golden Gate Bridge, Highway and Transportation...",major_stop_ferry,43007,TBSF,major_stop_ferry_single_operator,None,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,None,recoX7qMhlPrgfuz3,None,2,POINT (-122.45592 37.87273)
26185,"Golden Gate Bridge, Highway and Transportation...",major_stop_ferry,43008,AISF,major_stop_ferry_single_operator,None,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,None,recoX7qMhlPrgfuz3,None,5,POINT (-122.39376 37.79653)


In [54]:
new_stops >> filter(_.hqta_type == 'major_stop_ferry', _.agency_primary.str.contains('Golden'))

,agency_primary,hqta_type,stop_id,route_id,hqta_details,agency_secondary,base64_url_primary,base64_url_secondary,org_id_primary,org_id_secondary,peak_trips,geometry
55389,"Golden Gate Bridge, Highway and Transportation...",major_stop_ferry,43000,LSSF,major_stop_ferry_single_operator,None,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,None,recoX7qMhlPrgfuz3,None,3,POINT (-122.39243 37.79592)
55390,"Golden Gate Bridge, Highway and Transportation...",major_stop_ferry,43000,TBSF,major_stop_ferry_single_operator,None,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,None,recoX7qMhlPrgfuz3,None,3,POINT (-122.39243 37.79592)
55391,"Golden Gate Bridge, Highway and Transportation...",major_stop_ferry,43000,SSSF,major_stop_ferry_single_operator,None,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,None,recoX7qMhlPrgfuz3,None,3,POINT (-122.39243 37.79592)
55392,"Golden Gate Bridge, Highway and Transportation...",major_stop_ferry,43003,SSSF,major_stop_ferry_single_operator,None,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,None,recoX7qMhlPrgfuz3,None,1,POINT (-122.47755 37.85617)
55393,"Golden Gate Bridge, Highway and Transportation...",major_stop_ferry,43004,LSSF,major_stop_ferry_single_operator,None,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,None,recoX7qMhlPrgfuz3,None,2,POINT (-122.50835 37.94471)
55394,"Golden Gate Bridge, Highway and Transportation...",major_stop_ferry,43007,TBSF,major_stop_ferry_single_operator,None,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,None,recoX7qMhlPrgfuz3,None,1,POINT (-122.45592 37.87273)
55395,"Golden Gate Bridge, Highway and Transportation...",major_stop_ferry,43008,TBSF,major_stop_ferry_single_operator,None,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,None,recoX7qMhlPrgfuz3,None,1,POINT (-122.39376 37.79653)
55396,"Golden Gate Bridge, Highway and Transportation...",major_stop_ferry,43008,SSSF,major_stop_ferry_single_operator,None,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,None,recoX7qMhlPrgfuz3,None,1,POINT (-122.39376 37.79653)
55397,"Golden Gate Bridge, Highway and Transportation...",major_stop_ferry,43008,AISF,major_stop_ferry_single_operator,None,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,None,recoX7qMhlPrgfuz3,None,1,POINT (-122.39376 37.79653)


In [55]:
new_stops >> filter(_.hqta_type == 'major_stop_rail', _.agency_primary.str.contains('Bay Area'))

,agency_primary,hqta_type,stop_id,route_id,hqta_details,agency_secondary,base64_url_primary,base64_url_secondary,org_id_primary,org_id_secondary,peak_trips,geometry
99044,San Francisco Bay Area Rapid Transit District,major_stop_rail,12TH,Yellow-S,major_stop_rail_single_operator,None,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,None,recoQLeNRISCKF8I0,None,23,POINT (-122.27159 37.80347)
99045,San Francisco Bay Area Rapid Transit District,major_stop_rail,12TH,Orange-S,major_stop_rail_single_operator,None,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,None,recoQLeNRISCKF8I0,None,23,POINT (-122.27159 37.80347)
99046,San Francisco Bay Area Rapid Transit District,major_stop_rail,12TH,Red-S,major_stop_rail_single_operator,None,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,None,recoQLeNRISCKF8I0,None,23,POINT (-122.27159 37.80347)
99047,San Francisco Bay Area Rapid Transit District,major_stop_rail,12TH,Yellow-N,major_stop_rail_single_operator,None,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,None,recoQLeNRISCKF8I0,None,23,POINT (-122.27159 37.80347)
99048,San Francisco Bay Area Rapid Transit District,major_stop_rail,12TH,Orange-N,major_stop_rail_single_operator,None,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,None,recoQLeNRISCKF8I0,None,23,POINT (-122.27159 37.80347)
...,...,...,...,...,...,...,...,...,...,...,...,...
99269,San Francisco Bay Area Rapid Transit District,major_stop_rail,WOAK,Green-N,major_stop_rail_single_operator,None,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,None,recoQLeNRISCKF8I0,None,29,POINT (-122.29514 37.80492)
99270,San Francisco Bay Area Rapid Transit District,major_stop_rail,WOAK,Red-S,major_stop_rail_single_operator,None,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,None,recoQLeNRISCKF8I0,None,29,POINT (-122.29514 37.80492)
99271,San Francisco Bay Area Rapid Transit District,major_stop_rail,WOAK,Yellow-S,major_stop_rail_single_operator,None,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,None,recoQLeNRISCKF8I0,None,29,POINT (-122.29514 37.80492)
99272,San Francisco Bay Area Rapid Transit District,major_stop_rail,WOAK,Blue-S,major_stop_rail_single_operator,None,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,None,recoQLeNRISCKF8I0,None,29,POINT (-122.29514 37.80492)


In [56]:
old_stops >> filter(_.hqta_type == 'major_stop_rail', _.agency_primary.str.contains('Bay Area'))

,agency_primary,hqta_type,stop_id,route_id,hqta_details,agency_secondary,base64_url_primary,base64_url_secondary,org_id_primary,org_id_secondary,peak_trips,geometry
52186,San Francisco Bay Area Rapid Transit District,major_stop_rail,12TH,Yellow-S,major_stop_rail_single_operator,None,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,None,recoQLeNRISCKF8I0,None,24,POINT (-122.27159 37.80347)
52187,San Francisco Bay Area Rapid Transit District,major_stop_rail,16TH,Yellow-N,major_stop_rail_single_operator,None,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,None,recoQLeNRISCKF8I0,None,30,POINT (-122.41975 37.76518)
52188,San Francisco Bay Area Rapid Transit District,major_stop_rail,19TH,Orange-N,major_stop_rail_single_operator,None,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,None,recoQLeNRISCKF8I0,None,25,POINT (-122.26872 37.80806)
52189,San Francisco Bay Area Rapid Transit District,major_stop_rail,24TH,Red-S,major_stop_rail_single_operator,None,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,None,recoQLeNRISCKF8I0,None,30,POINT (-122.41852 37.75242)
52190,San Francisco Bay Area Rapid Transit District,major_stop_rail,ANTC,Yellow-N,major_stop_rail_single_operator,None,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,None,recoQLeNRISCKF8I0,None,6,POINT (-121.78035 37.99541)
52191,San Francisco Bay Area Rapid Transit District,major_stop_rail,ASHB,Orange-N,major_stop_rail_single_operator,None,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,None,recoQLeNRISCKF8I0,None,12,POINT (-122.26975 37.85307)
52192,San Francisco Bay Area Rapid Transit District,major_stop_rail,BALB,Red-S,major_stop_rail_single_operator,None,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,None,recoQLeNRISCKF8I0,None,30,POINT (-122.44751 37.72176)
52193,San Francisco Bay Area Rapid Transit District,major_stop_rail,BAYF,Green-S,major_stop_rail_single_operator,None,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,None,recoQLeNRISCKF8I0,None,18,POINT (-122.12643 37.69694)
52194,San Francisco Bay Area Rapid Transit District,major_stop_rail,BERY,Orange-N,major_stop_rail_single_operator,None,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,None,recoQLeNRISCKF8I0,None,12,POINT (-121.87466 37.36849)
52195,San Francisco Bay Area Rapid Transit District,major_stop_rail,CAST,Blue-S,major_stop_rail_single_operator,None,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,None,recoQLeNRISCKF8I0,None,6,POINT (-122.07561 37.69078)


In [58]:
new_stops

,agency_primary,hqta_type,stop_id,route_id,hqta_details,agency_secondary,base64_url_primary,base64_url_secondary,org_id_primary,org_id_secondary,peak_trips,geometry
0,Alameda-Contra Costa Transit District,hq_corridor_bus,50000,29,corridor_other_stop,None,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,None,recOZgevYf7Jimm9L,None,2,POINT (-122.24686 37.80941)
1,Alameda-Contra Costa Transit District,hq_corridor_bus,50006,L,corridor_other_stop,None,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,None,recOZgevYf7Jimm9L,None,0,POINT (-122.39791 37.78818)
2,Alameda-Contra Costa Transit District,hq_corridor_bus,50008,LA,corridor_other_stop,None,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,None,recOZgevYf7Jimm9L,None,0,POINT (-122.39776 37.78830)
3,Alameda-Contra Costa Transit District,hq_corridor_bus,50010,FS,corridor_other_stop,None,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,None,recOZgevYf7Jimm9L,None,0,POINT (-122.39761 37.78842)
4,Alameda-Contra Costa Transit District,hq_corridor_bus,50012,G,corridor_other_stop,None,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,None,recOZgevYf7Jimm9L,None,0,POINT (-122.39746 37.78854)
...,...,...,...,...,...,...,...,...,...,...,...,...
114832,Yuba-Sutter Transit Authority,major_stop_bus,758936,364,intersection_2_bus_routes_same_operator,Yuba-Sutter Transit Authority,aHR0cHM6Ly9kYXRhLnRyaWxsaXVtdHJhbnNpdC5jb20vZ3...,aHR0cHM6Ly9kYXRhLnRyaWxsaXVtdHJhbnNpdC5jb20vZ3...,rech0BusKA17h0vuL,rech0BusKA17h0vuL,3,POINT (-121.53724 39.07871)
114833,Yuma County Intergovernmental Public Transport...,hq_corridor_bus,43,23,corridor_other_stop,None,aHR0cHM6Ly93d3cueWNpcHRhLm9yZy9ndGZzL2dvb2dsZV...,None,reclW3IXkAAyivJtq,None,0,POINT (-114.61527 32.73178)
114834,Yuma County Intergovernmental Public Transport...,hq_corridor_bus,500,23,corridor_other_stop,None,aHR0cHM6Ly93d3cueWNpcHRhLm9yZy9ndGZzL2dvb2dsZV...,None,reclW3IXkAAyivJtq,None,0,POINT (-115.56774 32.79162)
114835,Yuma County Intergovernmental Public Transport...,hq_corridor_bus,536,23,corridor_other_stop,None,aHR0cHM6Ly93d3cueWNpcHRhLm9yZy9ndGZzL2dvb2dsZV...,None,reclW3IXkAAyivJtq,None,0,POINT (-115.55900 32.79160)
